# Exploratory Data Analysis with python and plotly

##### Contents

[1. Load data](###1.-Load-data)

[2. Data Understanding and Cleaning](###2.-data-understanding-and-cleaning)

### 1. Load data

In [1129]:

import pandas as pd

train = pd.read_csv('../data/raw/train.csv') 
test = pd.read_csv('../data/raw/test.csv')


Stick training and test data together so that feature engineering is done to the test set as well. 

In [1130]:
train['source'] = 'train'
test['source'] = 'test'
combined = pd.concat([train, test], sort=False)

### 2. Data Understanding and Cleaning

#### Understanding

In [1131]:
combined.head(10)



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,source
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train
5,6,0.0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,train
6,7,0.0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,train
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,train
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,train
9,10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,train


In [1132]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  source       1309 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 143.2+ KB


All of these datatypes make sense, none need to be converted

In [1133]:
combined.describe().round(2)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.00,891.00,1309.00,1046.00,1309.00,1309.00,1308.00
mean,655.00,0.38,2.29,29.88,0.50,0.39,33.30
std,378.02,0.49,0.84,14.41,1.04,0.87,51.76
min,1.00,0.00,1.00,0.17,0.00,0.00,0.00
25%,328.00,0.00,2.00,21.00,0.00,0.00,7.90
50%,655.00,0.00,3.00,28.00,0.00,0.00,14.45
75%,982.00,1.00,3.00,39.00,1.00,0.00,31.28
max,1309.00,1.00,3.00,80.00,8.00,9.00,512.33


#### Deal with Missing Values

In [1134]:
missing_values = combined.isnull().sum()
missing_values = missing_values[missing_values > 0]
print("Missing values in train data:")
print(missing_values)

Missing values in train data:
Survived     418
Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64


possible values

In [1135]:
unique_values = combined.nunique()
print(unique_values)

for col in combined.columns:
    if combined[col].nunique() < 10:
        print(f"{col}: {combined[col].unique()}") # show unique values for columns with less than 10 unique values


PassengerId    1309
Survived          2
Pclass            3
Name           1307
Sex               2
Age              98
SibSp             7
Parch             8
Ticket          929
Fare            281
Cabin           186
Embarked          3
source            2
dtype: int64
Survived: [ 0.  1. nan]
Pclass: [3 1 2]
Sex: ['male' 'female']
SibSp: [1 0 3 4 2 5 8]
Parch: [0 1 2 5 3 4 6 9]
Embarked: ['S' 'C' 'Q' nan]
source: ['train' 'test']


##### Embarked Column

In [1136]:
print(combined['Embarked'].value_counts())
print(combined['Embarked'].value_counts() / len(combined))


Embarked
S    914
C    270
Q    123
Name: count, dtype: int64
Embarked
S    0.698243
C    0.206264
Q    0.093965
Name: count, dtype: float64


Becuase there is a 73% chance a passanger embarked in Cherbourg (C), it makes sense to fill in the two missing values in with C

In [1137]:
print("Before filling:")
print(combined['Embarked'].isnull().sum()) # 2

# Fill missing values
combined['Embarked'] = combined['Embarked'].fillna(combined['Embarked'].mode()[0])

print("\nAfter filling:")
print(combined['Embarked'].isnull().sum()) # should be 0

Before filling:
2

After filling:
0


##### Age Column

In [1138]:
print(missing_values[0:1] / len(combined))

print(combined['Age'].describe().round(2))

Survived    0.319328
dtype: float64
count    1046.00
mean       29.88
std        14.41
min         0.17
25%        21.00
50%        28.00
75%        39.00
max        80.00
Name: Age, dtype: float64


For age, we have 20% of rows missing an entry. This is probably an important predictive columns so we we will impute values. 

First lets extract a title, beucase that coule be useful. 

In [1139]:
# add a nnew column for the title, found in the second index after splitting the name column
combined['Title'] = combined['Name'].str.split(',').str[1].str.split('.').str[0]

# print the unique values in the title column and the number of occurences
print(combined['Title'].value_counts())



Title
Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: count, dtype: int64


Predict the missing vlaues of age using XGBoost

In [1140]:
# lets use a Gradient Boosting Regressor to predict the missing values of age 
# using columns Pclass, Sex, SibSp, Parch, Embarked, Title.

import xgboost as xgb
import numpy as np
from sklearn.preprocessing import LabelEncoder

# First, let's prepare the data for XGBoost
# Create a copy of the data we'll use for prediction
age_data = combined[['Age', 'Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked', 'Title', 'Fare']].copy()

# Encode categorical variables
le_sex = LabelEncoder()
le_emb = LabelEncoder()
le_title = LabelEncoder()

age_data['Sex'] = le_sex.fit_transform(age_data['Sex'])
age_data['Embarked'] = le_emb.fit_transform(age_data['Embarked'])
age_data['Title'] = le_title.fit_transform(age_data['Title'])

# Split into two sets: known age and unknown age
known_age = age_data[age_data['Age'].notnull()]
unknown_age = age_data[age_data['Age'].isnull()]

# Prepare the training data
X_train = known_age.drop('Age', axis=1)
y_train = known_age['Age']

# Train XGBoost model
xgb_reg = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_reg.fit(X_train, y_train)

# Predict missing ages
X_test = unknown_age.drop('Age', axis=1)
predicted_ages = xgb_reg.predict(X_test)

# Fill the missing values in the original dataframe
combined.loc[combined['Age'].isnull(), 'Age'] = predicted_ages

# Verify the results
print("\nMissing values after prediction:")
print(combined['Age'].isnull().sum())

# Show age distribution statistics before and after
print("\nAge statistics after imputation:")
print(combined['Age'].describe().round(2))




Missing values after prediction:
0

Age statistics after imputation:
count    1309.00
mean       29.90
std        13.52
min         0.17
25%        21.96
50%        28.12
75%        38.00
max        80.00
Name: Age, dtype: float64


##### Cabin Column

For cabin we have two components, the letter and the number. Lets split these, creating new columns and deleting the old ones.

note that for some tickets that are booked together, there are multiple Cabins recoded, for these we can only take the cabin letter.

In [1141]:
def extract_cabin_info(cabin_entry):
    if pd.isna(cabin_entry):
        return pd.Series([np.nan, np.nan])
    cabins = cabin_entry.split()
    first_cabin = cabins[0]
    cabin_letter = first_cabin[0]
    cabin_number = first_cabin[1:] if len(cabins) == 1 else np.nan
    return pd.Series([cabin_letter, cabin_number])

combined[['Cabin_letter', 'Cabin_number']] = combined['Cabin'].apply(extract_cabin_info)

combined = combined.drop('Cabin', axis=1)

combined.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,source,Title,Cabin_letter,Cabin_number
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,train,Mr,NaN,NaN
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,train,Mrs,C,85
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,train,Miss,NaN,NaN
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,train,Mrs,C,123
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,train,Mr,NaN,NaN


In [1142]:

print(missing_values[1:2] / len(combined))

Age    0.200917
dtype: float64


77% of the Cabin letter and number were not recorded. This is too much to drop, and hard to predict given our other predictors. We will leave it as NaN.

### 3. Feature Engineering

#### Last Name

In [1143]:
combined['LastName'] = combined['Name'].str.split(',').str[0].str.split('.').str[0]
print(combined['LastName'].value_counts())

LastName
Andersson    11
Sage         11
Goodwin       8
Asplund       8
Davies        7
             ..
Milling       1
Maisner       1
Goncalves     1
Campbell      1
Saether       1
Name: count, Length: 875, dtype: int64


#### Family Size

In [1144]:
# Create family size column
combined['FamilySize'] = combined['SibSp'] + combined['Parch'] + 1  # +1 to include the passenger themselves

# Sort by FamilySize and then by Fare to help identify family groups
families = combined.loc[combined['FamilySize'] > 1].sort_values(
    ['FamilySize', 'LastName', 'Ticket', 'Fare'], 
    ascending=[False, False, True, True]
)

families.head(50)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,source,Title,Cabin_letter,Cabin_number,LastName,FamilySize
159,160,0.0,3,"Sage, Master. Thomas Henry",male,15.969364,8,2,CA. 2343,69.5500,S,train,Master,NaN,NaN,Sage,11
180,181,0.0,3,"Sage, Miss. Constance Gladys",female,23.197212,8,2,CA. 2343,69.5500,S,train,Miss,NaN,NaN,Sage,11
201,202,0.0,3,"Sage, Mr. Frederick",male,20.659479,8,2,CA. 2343,69.5500,S,train,Mr,NaN,NaN,Sage,11
324,325,0.0,3,"Sage, Mr. George John Jr",male,20.659479,8,2,CA. 2343,69.5500,S,train,Mr,NaN,NaN,Sage,11
792,793,0.0,3,"Sage, Miss. Stella Anna",female,23.197212,8,2,CA. 2343,69.5500,S,train,Miss,NaN,NaN,Sage,11
846,847,0.0,3,"Sage, Mr. Douglas Bullen",male,20.659479,8,2,CA. 2343,69.5500,S,train,Mr,NaN,NaN,Sage,11
863,864,0.0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,23.197212,8,2,CA. 2343,69.5500,S,train,Miss,NaN,NaN,Sage,11
188,1080,NaN,3,"Sage, Miss. Ada",female,23.197212,8,2,CA. 2343,69.5500,S,test,Miss,NaN,NaN,Sage,11
342,1234,NaN,3,"Sage, Mr. John George",male,39.557892,1,9,CA. 2343,69.5500,S,test,Mr,NaN,NaN,Sage,11
360,1252,NaN,3,"Sage, Master. William Henry",male,14.500000,8,2,CA. 2343,69.5500,S,test,Master,NaN,NaN,Sage,11


In [1145]:
combined.isnull().sum()

# print rows with missing vlaues in LastName_encoded




PassengerId        0
Survived         418
Pclass             0
Name               0
Sex                0
Age                0
SibSp              0
Parch              0
Ticket             0
Fare               1
Embarked           0
source             0
Title              0
Cabin_letter    1014
Cabin_number    1055
LastName           0
FamilySize         0
dtype: int64

Sweet that kind of works, its problems is that parents will (moslty) show up as having no parents on board, so will have incorrect family sizes. The family size will only work for the rows of children.

Note in the Sage's case there are parents and grandparents, so grandparents will not have the correct family size.

#### LastName

The last name is currently our most effective method for grouping families. However, due to its high cardinality, we will encode this feature using target encoding by assigning the mean survival rate to a new variable, LastName_encoded

In [1146]:
combined["LastName_encoded"] = combined.groupby("LastName")["Survived"].transform("mean")

combined[["LastName", "Survived", "LastName_encoded"]].head(10)

LName_stats = combined.groupby("LastName_encoded").agg(
    Survived_Mean=("Survived", "mean"),
    Count=("Survived", "count")
).sort_values(by="Count", ascending=False)

print(LName_stats)

                  Survived_Mean  Count
LastName_encoded                      
0.000000               0.000000    470
1.000000               1.000000    259
0.500000               0.500000     72
0.666667               0.666667     36
0.333333               0.333333     21
0.750000               0.750000     16
0.222222               0.222222      9
0.250000               0.250000      8


In [1147]:
combined = combined.drop('LastName', axis=1)

#### Title

We will also do target encoding with Title due to its high cartinality

In [1148]:
combined["Title_encoded"] = combined.groupby("Title")["Survived"].transform("mean")

title_stats = combined.groupby("Title").agg(
    Survived_Mean=("Survived", "mean"),
    Count=("Survived", "count")
).sort_values(by="Count", ascending=False)

print(title_stats)

              Survived_Mean  Count
Title                             
Mr                 0.156673    517
Miss               0.697802    182
Mrs                0.792000    125
Master             0.575000     40
Dr                 0.428571      7
Rev                0.000000      6
Major              0.500000      2
Col                0.500000      2
Mlle               1.000000      2
Sir                1.000000      1
Ms                 1.000000      1
Capt               0.000000      1
Mme                1.000000      1
Lady               1.000000      1
Jonkheer           0.000000      1
Don                0.000000      1
the Countess       1.000000      1
Dona                    NaN      0


In [1149]:
combined = combined.drop('Title', axis=1)

#### Drop columns with high cartinality or non-useful Information

In [1150]:
# Calculate original cardinality and data types
cardinality_before = combined.nunique()
dtypes_before = combined.dtypes

# Combine into a DataFrame
cardinality_df_before = pd.DataFrame({
    'Cardinality': cardinality_before,
    'Dtype': dtypes_before
})

print("Cardinality and Data Types (Before Dropping):")
print(cardinality_df_before.sort_values(by="Cardinality", ascending=False))
print("\n")

# Identify high-cardinality columns
high_cardinality = cardinality_before[cardinality_before > 10].index

# Drop only if dtype is not float
columns_to_drop = [col for col in high_cardinality if not pd.api.types.is_float_dtype(combined[col])]

print("Columns to drop (cardinality > 10 and not float):")
for col in columns_to_drop:
    print(f"{col}: {combined[col].dtype}")
# drop the Cabin_
print("\n")

# Drop from train and test
combined.drop(columns=columns_to_drop, inplace=True)

# Calculate new cardinality and data types
cardinality_after = combined.nunique()
dtypes_after = combined.dtypes

# Combine into a DataFrame
cardinality_df_after = pd.DataFrame({
    'Cardinality': cardinality_after,
    'Dtype': dtypes_after
})

print("Cardinality and Data Types (After Dropping):")
print(cardinality_df_after.sort_values(by="Cardinality", ascending=False))

Cardinality and Data Types (Before Dropping):
                  Cardinality    Dtype
PassengerId              1309    int64
Name                     1307   object
Ticket                    929   object
Fare                      281  float64
Age                       224  float64
Cabin_number              101   object
FamilySize                  9    int64
Parch                       8    int64
Cabin_letter                8   object
LastName_encoded            8  float64
Title_encoded               8  float64
SibSp                       7    int64
Pclass                      3    int64
Embarked                    3   object
Sex                         2   object
Survived                    2  float64
source                      2   object


Columns to drop (cardinality > 10 and not float):
PassengerId: int64
Name: object
Ticket: object
Cabin_number: object


Cardinality and Data Types (After Dropping):
                  Cardinality    Dtype
Fare                      281  float64
Age    

#### Cabin_letter

In [1151]:
# Step 2: Replace NaNs with 'Missing'
combined['Cabin_letter'] = combined['Cabin_letter'].fillna('Missing')

# Step 3: Calculate target mean encoding
combined["Cabin_letter_encoded"] = combined.groupby("Cabin_letter")["Survived"].transform("mean")

# Optional: Create stats summary
Cabin_stats = combined.groupby("Cabin_letter").agg(
    Survived_Mean=("Survived", "mean"),
    Count=("Survived", "count")
).sort_values(by="Count", ascending=False)

print(Cabin_stats)

              Survived_Mean  Count
Cabin_letter                      
Missing            0.299854    687
C                  0.593220     59
B                  0.744681     47
D                  0.757576     33
E                  0.750000     32
A                  0.466667     15
F                  0.615385     13
G                  0.500000      4
T                  0.000000      1


In [1152]:
# print average survival rate overall
print(combined["Survived"].mean())

0.3838383838383838


In [1153]:

# drop rows with one missing value

In [1154]:
missing_values = combined.isnull().sum()
print("Missing values in train data:")
print(missing_values)

Missing values in train data:
Survived                418
Pclass                    0
Sex                       0
Age                       0
SibSp                     0
Parch                     0
Fare                      1
Embarked                  0
source                    0
Cabin_letter              0
FamilySize                0
LastName_encoded        230
Title_encoded             1
Cabin_letter_encoded      0
dtype: int64


In [1155]:
combined = combined.dropna(subset=['Title_encoded'])
combined = combined.dropna(subset=['Fare'])
combined = combined.dropna(subset=['LastName_encoded'])


### 4. Visual Exploatory Analysis

In [1156]:
fig = px.histogram(combined, x='Age', color='Pclass', barmode='overlay', 
                   facet_col='Sex',
                   title='Age Distribution by Passenger Class and Sex',
                   opacity=0.6)
fig.show()

In [1157]:
fig = px.histogram(combined, x='Cabin_letter', color='Pclass', title='Passengers by Cabin Letter')
fig.show()

In [1158]:
combined = combined.drop('Cabin_letter', axis=1)

In [1159]:
fig = px.box(combined, x='Embarked', y='Fare', color='Embarked', title='Fare by Embarkation Port')
fig.show()


In [1160]:
fig = px.box(combined, x='Pclass', y='Fare', points='all', title='Fare Distribution by Passenger Class')
fig.show()

### Normalise

In [1161]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
cols_to_scale = ['Pclass', 'SibSp', 'Parch', 'Age', 'Fare', 'FamilySize', 'LastName_encoded', 'Title_encoded', 'Cabin_letter_encoded']

combined[cols_to_scale] = scaler.fit_transform(combined[cols_to_scale])

In [1162]:
missing_values = combined.isnull().sum()
print("Missing values in combined data:")
print(missing_values)

Missing values in combined data:
Survived                188
Pclass                    0
Sex                       0
Age                       0
SibSp                     0
Parch                     0
Fare                      0
Embarked                  0
source                    0
FamilySize                0
LastName_encoded          0
Title_encoded             0
Cabin_letter_encoded      0
dtype: int64


### 5. Export Cleaned Data

In [1163]:
train = combined[combined['source'] == 'train'].drop(columns=['source'])
test = combined[combined['source'] == 'test'].drop(columns=['source'])

In [1164]:
train.to_csv('../data/processed/train_cleaned.csv', index=False)
test.to_csv('../data/processed/test_cleaned.csv', index=False)

In [1165]:
combined.sort_values(by=['source', 'LastName_encoded'], ascending=[True, True]).head(400)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,source,FamilySize,LastName_encoded,Title_encoded,Cabin_letter_encoded
5,NaN,0.831128,male,-1.120477,-0.518045,-0.493553,-0.474047,S,test,-0.614088,-0.898412,-0.855763,-0.521637
14,NaN,-1.560305,female,1.248129,0.376308,-0.493553,0.499087,S,test,-0.023008,-0.898412,1.377861,2.247097
15,NaN,-0.364588,female,-0.402718,0.376308,-0.493553,-0.127581,C,test,-0.023008,-0.898412,1.377861,-0.521637
18,NaN,0.831128,female,-0.187390,0.376308,-0.493553,-0.498399,S,test,-0.023008,-0.898412,1.046689,-0.521637
21,NaN,0.831128,male,-1.479357,-0.518045,0.580124,-0.587455,S,test,-0.023008,-0.898412,0.614952,-0.521637
...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,0.0,-0.364588,male,-0.474494,-0.518045,-0.493553,-0.450164,S,train,-0.614088,-0.898412,0.100152,-0.521637
401,0.0,0.831128,male,-0.259166,-0.518045,-0.493553,-0.496057,S,train,-0.614088,-0.898412,-0.855763,-0.521637
404,0.0,0.831128,female,-0.689822,-0.518045,-0.493553,-0.484584,S,train,-0.614088,-0.898412,1.046689,-0.521637
405,0.0,-0.364588,male,0.315041,0.376308,-0.493553,-0.253476,S,train,-0.023008,-0.898412,-0.855763,-0.521637


In [1166]:
missing_values = combined.isnull().sum()
print("Missing values in train data:")
print(missing_values)

test.head(50)

Missing values in train data:
Survived                188
Pclass                    0
Sex                       0
Age                       0
SibSp                     0
Parch                     0
Fare                      0
Embarked                  0
source                    0
FamilySize                0
LastName_encoded          0
Title_encoded             0
Cabin_letter_encoded      0
dtype: int64


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,LastName_encoded,Title_encoded,Cabin_letter_encoded
0,NaN,0.831128,male,0.350929,-0.518045,-0.493553,-0.500193,Q,-0.614088,0.778298,-0.855763,-0.521637
4,NaN,0.831128,female,-0.546270,0.376308,0.580124,-0.416680,S,0.568072,1.337201,1.377861,-0.521637
5,NaN,0.831128,male,-1.120477,-0.518045,-0.493553,-0.474047,S,-0.614088,-0.898412,-0.855763,-0.521637
6,NaN,0.831128,female,0.027938,-0.518045,-0.493553,-0.503940,Q,-0.614088,1.337201,1.046689,-0.521637
7,NaN,-0.364588,male,-0.259166,0.376308,0.580124,-0.103619,S,0.568072,1.337201,-0.855763,-0.521637
9,NaN,0.831128,male,-0.618046,1.270660,-0.493553,-0.194470,S,0.568072,-0.153208,-0.855763,-0.521637
14,NaN,-1.560305,female,1.248129,0.376308,-0.493553,0.499087,S,-0.023008,-0.898412,1.377861,2.247097
15,NaN,-0.364588,female,-0.402718,0.376308,-0.493553,-0.127581,C,-0.023008,-0.898412,1.377861,-0.521637
16,NaN,-0.364588,male,0.386817,-0.518045,-0.493553,-0.415509,Q,-0.614088,0.219394,-0.855763,-0.521637
18,NaN,0.831128,female,-0.187390,0.376308,-0.493553,-0.498399,S,-0.023008,-0.898412,1.046689,-0.521637
